# Initial Data Exploration
## Ticket Classification Dataset Analysis

**Objective**: Explore the structure and basic characteristics of the ticket dataset
- Dataset: 6,968 tickets with extensive text fields
- Challenge: Multi-line CSV due to extensive Comments/Work Notes and Description fields

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)

# File path - use absolute path
data_path = Path('/Users/sparshk/Documents/Development/Ticket Classification/data/raw/actual_raw_data.csv')
print(f"File exists: {data_path.exists()}")
if data_path.exists():
    print(f"File size: {data_path.stat().st_size / (1024*1024):.1f} MB")
else:
    print("ERROR: File not found! Please check the path.")

File exists: True
File size: 42.4 MB


## Step 1: Initial File Examination
Let's first understand the file structure and handle the multi-line CSV properly

In [4]:
# First, let's examine the first few lines to understand structure
with open(data_path, 'r', encoding='utf-8') as f:
    first_lines = [f.readline().strip() for _ in range(10)]
    
print("First 10 lines of the file:")
for i, line in enumerate(first_lines, 1):
    print(f"{i:2d}: {line[:100]}{'...' if len(line) > 100 else ''}")

First 10 lines of the file:
 1: ﻿Short description,Category,Priority,Assignment group,Subcategory,Vendor,Comments and Work notes,Des...
 2: TILL: Tills are down,Server,1 - Critical,WKS-HEX-Server-Support,Windows Server,JBrand,"2025-01-24 19...
 3: Incident automatically closed by system after 7 days in the Resolved state.
 4: 
 5: 2025-01-17 18:20:21 - Vinoth Kumar (Work notes)
 6: Knowledge article[code]<a href=kb_view.do?sys_kb_id=ac0bd5bd9790c2501049b0d3f153af0c>KB0010949</a>[/...
 7: Version: 1
 8: 
 9: 2025-01-17 16:22:15 - Kuppanagari Maghana (Additional comments)
10: Hi Cindy,


In [5]:
# Count total lines in file
with open(data_path, 'r', encoding='utf-8') as f:
    total_lines = sum(1 for _ in f)

print(f"Total lines in file: {total_lines:,}")
print(f"Expected records if ~6,968 tickets: {6968}")
print(f"Average lines per record: {total_lines / 6968:.1f}")

Total lines in file: 519,107
Expected records if ~6,968 tickets: 6968
Average lines per record: 74.5


## Step 2: Load Data with Proper CSV Handling
Handle the multi-line nature of the CSV due to extensive text fields

In [6]:
# Try loading with different CSV parameters to handle multi-line fields
try:
    # Load with quoting to handle multi-line fields properly
    df = pd.read_csv(data_path, 
                     encoding='utf-8',
                     quoting=1,  # QUOTE_ALL
                     skipinitialspace=True,
                     on_bad_lines='warn')
    
    print(f"Successfully loaded DataFrame with shape: {df.shape}")
    print(f"Columns: {list(df.columns)}")
    
except Exception as e:
    print(f"Error loading with standard method: {e}")
    print("Trying alternative loading method...")
    
    # Alternative: try with different parameters
    df = pd.read_csv(data_path, 
                     encoding='utf-8',
                     quotechar='"',
                     skipinitialspace=True,
                     on_bad_lines='skip')
    
    print(f"Loaded with alternative method. Shape: {df.shape}")

Successfully loaded DataFrame with shape: (6968, 8)
Columns: ['Short description', 'Category', 'Priority', 'Assignment group', 'Subcategory', 'Vendor', 'Comments and Work notes', 'Description']


In [7]:
# Basic info about the dataset
print("=== DATASET OVERVIEW ===")
print(f"Shape: {df.shape}")
print(f"Columns: {len(df.columns)}")
print("\nColumn names:")
for i, col in enumerate(df.columns, 1):
    print(f"{i:2d}. {col}")

=== DATASET OVERVIEW ===
Shape: (6968, 8)
Columns: 8

Column names:
 1. Short description
 2. Category
 3. Priority
 4. Assignment group
 5. Subcategory
 6. Vendor
 7. Comments and Work notes
 8. Description


In [8]:
# Check data types and basic statistics
print("=== DATA TYPES ===")
print(df.dtypes)
print("\n=== MEMORY USAGE ===")
print(df.info(memory_usage='deep'))

=== DATA TYPES ===
Short description          object
Category                   object
Priority                   object
Assignment group           object
Subcategory                object
Vendor                     object
Comments and Work notes    object
Description                object
dtype: object

=== MEMORY USAGE ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6968 entries, 0 to 6967
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Short description        6968 non-null   object
 1   Category                 6968 non-null   object
 2   Priority                 6968 non-null   object
 3   Assignment group         6968 non-null   object
 4   Subcategory              6968 non-null   object
 5   Vendor                   508 non-null    object
 6   Comments and Work notes  6968 non-null   object
 7   Description              6964 non-null   object
dtypes: object(8)
memory usage: 56.1 

## Step 3: Examine Sample Records
Look at a few complete records to understand the data structure

In [9]:
# Display first few records
print("=== FIRST 3 RECORDS ===")
for i in range(min(3, len(df))):
    print(f"\n--- RECORD {i+1} ---")
    for col in df.columns:
        value = str(df.iloc[i][col]) if pd.notna(df.iloc[i][col]) else "[NULL]"
        # Truncate very long values
        if len(value) > 200:
            value = value[:200] + "...[TRUNCATED]"
        print(f"{col}: {value}")

=== FIRST 3 RECORDS ===

--- RECORD 1 ---
Short description: TILL: Tills are down
Category: Server
Priority: 1 - Critical
Assignment group: WKS-HEX-Server-Support
Subcategory: Windows Server
Vendor: JBrand
Comments and Work notes: 2025-01-24 19:00:00 - System (Additional comments)
Incident automatically closed by system after 7 days in the Resolved state.

2025-01-17 18:20:21 - Vinoth Kumar (Work notes)
Knowledge article[code]<...[TRUNCATED]
Description: Description Of the Issue:  All tills are down, from the morning tills were working slow
Impacted Location / Store:  8501
Store Manger Name:  Cindy Skingley
Manager Contact Number: +44 7852 953927
What...[TRUNCATED]

--- RECORD 2 ---
Short description: Till/Backoffice/Zebra/ Phones - Down 
Category: Network
Priority: 1 - Critical
Assignment group: WKS-HEX-NWDATA-Support
Subcategory: Network Down
Vendor: [NULL]
Comments and Work notes: 2025-01-20 14:00:01 - System (Additional comments)
Incident automatically closed by system after 7 days

## Step 4: Basic Statistics and Missing Values

In [10]:
# Missing values analysis
print("=== MISSING VALUES ANALYSIS ===")
missing_stats = pd.DataFrame({
    'Column': df.columns,
    'Missing Count': df.isnull().sum(),
    'Missing %': (df.isnull().sum() / len(df) * 100).round(2),
    'Data Type': df.dtypes
})

print(missing_stats.sort_values('Missing %', ascending=False))

=== MISSING VALUES ANALYSIS ===
                                          Column  Missing Count  Missing %  \
Vendor                                    Vendor           6460      92.71   
Description                          Description              4       0.06   
Short description              Short description              0       0.00   
Category                                Category              0       0.00   
Priority                                Priority              0       0.00   
Assignment group                Assignment group              0       0.00   
Subcategory                          Subcategory              0       0.00   
Comments and Work notes  Comments and Work notes              0       0.00   

                        Data Type  
Vendor                     object  
Description                object  
Short description          object  
Category                   object  
Priority                   object  
Assignment group           object  
Subcategory  

In [11]:
# Text field length analysis for the extensive fields
print("=== TEXT FIELD LENGTH ANALYSIS ===")

text_fields = ['Comments and Work notes', 'Description', 'Short description']

for field in text_fields:
    if field in df.columns:
        # Calculate lengths (handle NaN values)
        lengths = df[field].astype(str).str.len()
        
        print(f"\n{field}:")
        print(f"  Min length: {lengths.min():,}")
        print(f"  Max length: {lengths.max():,}")
        print(f"  Mean length: {lengths.mean():.0f}")
        print(f"  Median length: {lengths.median():.0f}")
        print(f"  95th percentile: {lengths.quantile(0.95):.0f}")

=== TEXT FIELD LENGTH ANALYSIS ===

Comments and Work notes:
  Min length: 127
  Max length: 32,002
  Mean length: 5848
  Median length: 1510
  95th percentile: 26520

Description:
  Min length: 3
  Max length: 3,425
  Mean length: 323
  Median length: 272
  95th percentile: 737

Short description:
  Min length: 4
  Max length: 160
  Mean length: 38
  Median length: 33
  95th percentile: 91


## Step 5: Initial Summary
Summarize key findings from this initial exploration

In [12]:
print("=== INITIAL EXPLORATION SUMMARY ===")
print(f"✓ Dataset loaded successfully")
print(f"✓ Shape: {df.shape[0]:,} records × {df.shape[1]} columns")
print(f"✓ Expected ~6,968 tickets: {'✓ MATCH' if abs(df.shape[0] - 6968) < 100 else '✗ MISMATCH'}")
print(f"✓ Memory usage: {df.memory_usage(deep=True).sum() / (1024*1024):.1f} MB")

print("\n=== NEXT STEPS ===")
print("1. Analyze structured columns (Category, Priority, Assignment group, etc.)")
print("2. Deep dive into extensive text fields")
print("3. Data quality assessment")
print("4. Distribution analysis")
print("5. Text preprocessing preparation")

=== INITIAL EXPLORATION SUMMARY ===
✓ Dataset loaded successfully
✓ Shape: 6,968 records × 8 columns
✓ Expected ~6,968 tickets: ✓ MATCH
✓ Memory usage: 56.1 MB

=== NEXT STEPS ===
1. Analyze structured columns (Category, Priority, Assignment group, etc.)
2. Deep dive into extensive text fields
3. Data quality assessment
4. Distribution analysis
5. Text preprocessing preparation
